In [1]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM

# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

# Embedding dimensions.
row_hidden = 128
col_hidden = 128

# The data, split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4D for Hierarchical RNN.

# HRNNs can learn across multiple levels of temporal hierarchy over a complex sequence. Usually, the first recurrent layer of an HRNN encodes a sentence
# (e.g. of word vectors) into a sentence vector. The second recurrent layer then encodes a sequence of such vectors (encoded by the first layer) into a 
# document vector. This document vector is considered to preserve both the word-level and sentence-level structure of the context.
 
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

In [3]:
x.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(1)])

In [4]:
# Encodes a row of pixels using TimeDistributed Wrapper.
# TimeDistributed. This wrapper applies a layer to every temporal slice of an input. The input should be at least 3D, and the dimension of index one will be 
# considered to be the temporal dimension. Consider a batch of 32 samples, where each sample is a sequence of 10 vectors of 16 dimensions

encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

In [5]:
# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 28, 128)           66560     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 199,434
Trainable params: 199,434
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 471s 8ms/step - loss: 0.5058 - acc: 0.8312 - val_loss: 0.1418 - val_acc: 0.9568
Epoch 2/5
60000/60000 [==============================] - 273s 5ms/step - loss: 0.1288 - acc: 0.9609 - val_loss: 0.0753 - val_acc: 0.9773
Epoch 3/5
60000/60000 [==============================] - 272s 5ms/step - loss: 0.0826 - acc: 0.9757 - val_loss: 0.0743 - val_acc: 0.9749
Epoch 4/5
60000/60000 [==============================] - 272s 5ms/step - loss: 0.0651 - acc: 0.9807 - val_loss: 0.0538 - val_acc: 0.9827
Epoch 5/5
60000/60000 [==============================] - 340s 6ms/step - loss: 0.0528 - acc: 0.9843 - val_loss: 0.0425 - val_acc: 0.9874
Test loss: 0.04252630284940824
Test accuracy: 0.9874
